In [1]:
import importlib
import dbops
import cv2
import time
import numpy as np
from tqdm import tqdm
from PIL import Image
from IPython.display import display

import torch

from facelib import FaceDetector, EmotionDetector
import glob
import math

/scratch/u1319464/miniforge3/envs/ds4150-faces/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
movie = dbops.Movie('tt15439366')
movie.get_title(), movie.get_file_path(), movie.get_date_created()

('The Invisible Man (1933)',
 './data/Invisible Man 1933.mp4',
 datetime.datetime(2018, 9, 25, 0, 0))

In [3]:

face_detector = FaceDetector()
emotion_detector = EmotionDetector()


from EmotionDetector: weights loaded


In [4]:

face_index_sample_resolution = 15 # seconds
process_frame_subsample_rate = 30 # 1 = process all frames, 2 = process every other frame, 3 = process every thrid frame
# to improve performance, we first build an index of every 15 seconds that tells us whether that frame had a face.
# next, we only process interfavals of 15 seconds which started and/or ended with a frame which had a face
# this avoids processing areas with no faces

num_faces_index = {} # frame_index --> number of faces

def process_frame(frame_ref: dbops.Frame, frame_array_bgr=None):

    if frame_array_bgr is None:
        raise Error('frame_array_bgr must be supplied, cache restore not implemented')
        # frame_array_bgr = np.array(frame_ref.get_frame_image())

    frame_ref.compute_frame()
    
    # Detect faces
    with torch.no_grad():
        aligned_faces, boxes, scores, landmarks = face_detector.detect_align(frame_array_bgr)

        num_faces = len(aligned_faces)
        frame_ref.set_num_faces(num_faces)
        
        num_faces_index[frame_ref.frame_index] = num_faces
    
        # Recognize facial expression (happy, sad, angry, etc.)
        # emotions, probab = emotion_detector.detect_emotion(faces)
        if num_faces>0:
            
            for i, (aligned_face, landmark) in enumerate(zip(aligned_faces.cpu().numpy(), landmarks.cpu().numpy())):
                face = dbops.Face.add_face(imdb_id, frame_idx, i)

                face.set_face_image_aligned(Image.fromarray(
                    cv2.cvtColor(aligned_face, cv2.COLOR_BGR2RGB).clip(0, 255).astype(np.uint8)
                ))
                face.set_facial_landmarks(landmark)
                face.compute_face()
        # else:
        #     print(f'No faces found in frame {frame_idx}')


for movie_ref in dbops.Movie.iterate_all_movies():
    imdb_id = movie_ref.get_imdb_id()
    title = movie_ref.get_title()
    file_path = movie_ref.get_file_path()
    
    print(f'Now Processing: {title}')

    video_stream = cv2.VideoCapture(file_path)

    if not video_stream.isOpened():
        print(f"Error: Could not open video {file_path}")
    
    fps = video_stream.get(cv2.CAP_PROP_FPS)       # frames per second

    offset_minutes = 10
    offset_in_frames = int(offset_minutes * 60 * fps)
    video_stream.set(cv2.CAP_PROP_POS_FRAMES, offset_in_frames)
    
    # --- PROCESS next 5 minutes of frames ---
    total_frames = int(video_stream.get(cv2.CAP_PROP_FRAME_COUNT))
    frames_to_process = total_frames - int(5 * 60 * fps)  # 5 minutes = 300 seconds
    
    # Ensure we don't exceed the total number of frames
    upper_frame_limit = offset_in_frames + frames_to_process
    if upper_frame_limit > total_frames:
        upper_frame_limit = total_frames
    
    print(f"Starting from minute {offset_minutes}, processing up to minute {(upper_frame_limit / 60 / fps):02f}")

    print(f'Building num_faces index of frames from every {face_index_sample_resolution} seconds')

    num_faces_index = {} # clear previous frames
    
    for frame_idx in tqdm(range(offset_in_frames, upper_frame_limit, int(math.ceil(face_index_sample_resolution * fps))), desc='Building Index'):
        frame_ref = dbops.Frame.add_frame(imdb_id, frame_idx)
        video_stream.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = video_stream.read()

        if not ret:
            print(f"Error: Could not read frame: {frame_idx}")
            continue

        # looks hackey but this avoids worring about caching the whole movie somewhere.
        # instead, the movie is streamed right here, and then we get rid of it after
        frame_ref.frame_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
        
        process_frame(frame_ref, frame)
        
        del frame_ref, frame

    processed_frames = movie_ref.iterate_frames()
    processed_frame_idxs = set( f.frame_index for f in movie_ref.iterate_frames() )

    print('Processing facial scenes')

    video_stream.set(cv2.CAP_PROP_POS_FRAMES, offset_in_frames) # restart video
    
    indexed_frames = list(sorted(num_faces_index.keys()))
    for i in tqdm(range(len(indexed_frames)-1), desc='Processing Frames'):
        
        start_frame = indexed_frames[i]
        end_frame =   indexed_frames[i+1]
        print(f'Start frame: {start_frame}\tEnd Frame: {end_frame}')
        
        if num_faces_index[start_frame] > 0 or num_faces_index[end_frame] > 0:

            # only compute frames in the current scene, given the downsample rate, and remove any frames which were already processed
            
            total_frames_in_scene = end_frame - start_frame
            frames_to_process = set(range(start_frame, end_frame, process_frame_subsample_rate))
            
            # total_frames_in_subsampled_scene = len(frames_to_process)
            frames_to_process = frames_to_process - processed_frame_idxs

            total_unprocessed_frames_in_subsampled_scene = len(frames_to_process)

            print(f'Computation reduction from downsampling & caching: {((total_frames_in_scene - total_unprocessed_frames_in_subsampled_scene) / total_frames_in_scene):02f}')
            
            for frame_idx in sorted(frames_to_process):
        
                frame_ref = dbops.Frame.add_frame(imdb_id, frame_idx)
        
                # if frame_ref.is_fully_cached(): # the above check is not full-proof like this but is way faster, and makes this redundant
                #     print(f'Skipped frame {frame_idx}')
                #     continue

                video_stream.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
                ret, frame = video_stream.read()
        
                if not ret:
                    print(f"Error: Could not read frame: {frame_idx}")
                    continue
        
                # looks hackey but this avoids worring about caching the whole movie somewhere.
                # instead, the movie is streamed right here, and then we get rid of it after
                frame_ref.frame_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

                process_frame(frame_ref, frame_array_bgr=frame)
                
                del frame_ref, frame

        else:
            print('No faces detected by index; Skipping scene.')

    video_stream.release()
print("Processing complete!")


Now Processing: Earth vs. the Flying Saucers
Starting from minute 10, processing up to minute 80.311333
Building num_faces index of frames from every 15 seconds


Building Index: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 282/282 [00:22<00:00, 12.55it/s]


Processing facial scenes


Processing Frames:   0%|                                                                                                                           | 0/281 [00:00<?, ?it/s]

Start frame: 14999	End Frame: 15374
No faces detected by index; Skipping scene.
Start frame: 15374	End Frame: 15749
Computation reduction from downsampling & caching: 0.968000


...

Processing Frames:  39%|███████████████████████████████████████████▌                                                                     | 105/272 [01:28<01:07,  2.46it/s]

Start frame: 52185	End Frame: 52545
Computation reduction from downsampling & caching: 0.969444


Processing Frames:  39%|████████████████████████████████████████████                                                                     | 106/272 [01:29<01:26,  1.91it/s]

Start frame: 52545	End Frame: 52905
Computation reduction from downsampling & caching: 0.969444


Processing Frames:  39%|████████████████████████████████████████████▍                                                                    | 107/272 [01:30<01:43,  1.59it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [5]:
# face_image_cache_key = f'{movie.imdb_id}_*.png'
# faces_output_mp4 = f'{movie.imdb_id}.mp4'
# !ffmpeg -framerate 30 -pattern_type glob -i $face_image_cache_key \
#     -c:v libx264 -pix_fmt yuv420p $faces_output_mp4

In [6]:
!ls images/face/ | head

tt0021814_000014385_000.png
tt0021814_000014415_000.png
tt0021814_000014445_000.png
tt0021814_000014475_000.png
tt0021814_000014505_000.png
tt0021814_000014535_000.png
tt0021814_000014565_000.png
tt0021814_000014595_000.png
tt0021814_000014625_000.png
tt0021814_000014655_000.png
ls: write error: Broken pipe


In [7]:
movie.get_imdb_id()

'tt15439366'

In [8]:
!du -sh movies.db omdb images data tmp

41M	movies.db
251K	omdb
1.2G	images
23G	data
du: cannot access 'tmp': No such file or directory


In [9]:
dbops.conn.close()
dbops.conn = dbops.psycopg2.connect("dbname=postgres user=u1319464 port=2111")
importlib.reload(dbops)  # Reload the module from source

<module 'dbops' from '/scratch/u1319464/Data-Mining-Spring-2025-Project/dbops.py'>